In [6]:
import requests
from urllib.parse import quote
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### 1. 대상지 주소 입력 > 법정동코드 추출

In [7]:
from urllib.parse import quote
with open('road_key.txt') as file:
    road_key = f'{file.read()}'

In [9]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote(input())}&resultType=json'
url = f"{road_url}?{params1}&{params2}"
result = requests.get(url).json()

BJD_code=result['results']['juso'][0]['admCd'] 
BJD_code

'4215034022'

### 2. 법정동코드 > 대상지 기후대, 토양성분 추출

In [12]:
#https://www.forest.go.kr/kfsweb/kfi/kfs/trail/soil.do?mn=NKFS_06_08_02&fileDownload=Y&dataType=/soil/FS_IJ100_50.zip&pblicDataId=PBD0000181&tabs=1&searchSrvc=&subTitle=&searchWrd=&searchCnd=
addrr = pd.read_csv('제주시.csv', sep=',')  #전국.csv만들기
addrr = pd.DataFrame(addrr)
addrr = addrr[['EMNDN_CD','CLZN_CD']]
addrr.columns = ['읍면동코드','기후대코드']
addrr['읍면동코드'] = int(BJD_code[:-2])
climate_code = addrr['기후대코드'].mode()[0]
climate_code  

# 1) 온대북부, 2)온대중부, 3)온대남부 4) 난대

# if climate_code == 1.0:
#     print('온대북부')
# elif climate_code == 2.0:
#     print('온대중부')
# elif climate_code == 3.0:
#     print('온대남부')
# elif climate_code == 4.0:
#     print('난대')

KeyError: "['EMNDN_CD'] not in index"

In [1]:
with open('soil_key.txt') as file:
    soilkey = file.read()

s_url = f'http://apis.data.go.kr/1390802/SoilEnviron/SoilExam/getSoilExamList?serviceKey={soilkey}&Page_Size=10&Page_No=1&BJD_Code={BJD_code}'
s_result = requests.get(s_url)
s_contents = s_result.text
s_soup = BeautifulSoup(s_contents, 'html.parser')

acid = []
vldpha = []
vldsia = []
om = []
posifert_mg = []
posifert_k = []
posifert_ca = []
selc= []

lis = [acid, vldpha, vldsia, om, posifert_mg, posifert_k, posifert_ca, selc]
cons = ['acid', 'vldpha', 'vldsia', 'om', 'posifert_mg', 'posifert_k', 'posifert_ca', 'selc']

for li in lis:
    for con in cons:
        if lis.index(li) == cons.index(con):
            for a in s_soup.find_all(con):
                li.append(float(a.get_text()))

ground_con = pd.DataFrame({'산도':acid, '유효인산':vldpha, '유효규산':vldsia, '유기물':om, '마그네슘':posifert_mg, '칼륨':posifert_k , '칼슘':posifert_ca, '전기전도도':selc})
ground_con

FileNotFoundError: [Errno 2] No such file or directory: 'gapikey.txt'